## 【实战】评估GSM8K数据集下模块的表现（优化篇）

使用 dspy 库，让一个大语言模型 (LLM) 变成一个能解答数学应用题的小天才。


想象一下，你有一个聪明的学生（大语言模型），我们想评估它做数学题能力并后续准备对它进行优化。

* **dspy:**  就像一个家教辅导机构，提供了一套教学方法和工具，让你的教学更高效。
* **GSM8K:**  就像一本习题册，里面有很多数学应用题。
* **`CoT` (Chain of Thought):** 这是一种教学方法，要求学生在解答问题时，一步步写出解题思路，就像我们平时做数学题时的“思考过程”。
* **`trainset, devset`:** 把习题册分成两部分，一部分用来教学生 (trainset)，一部分用来考试 (devset)。
* **`evaluate(...)`:**  最后，用考试题来检验学生的学习成果。



当你已经利用DSPy对系统做出编程与评估后，接下来就可以使用DSPy的优化器（optimizers）来微调你的程序中使用的提示（prompts）或权重（weights）。这一阶段很重要，此时你可能需要从仅有的探索数据集（development set）扩展到建立完整的数据集结构，包括：
- **训练集**（trainset）: 用于优化模型参数（提示或权重）的数据。优化器会尝试在训练集上找到最佳的参数配置。
- **验证集**（validation set）: 从训练集中分割出的一部分数据，用于在优化过程中选择最佳模型配置（在深度神经网络中这种做法很常见，但在这里的优化中提示与训练/验证比例可以与常规方法不同）。用于在优化过程中监控模型性能，防止过拟合。优化器会在训练集上训练，然后在验证集上评估，选择在验证集上表现最好的参数配置作为最后的优化结果。
- **开发集**（devset）: 用于最终评估模型性能的数据。测试集应该与训练集和验证集完全独立，以确保评估结果的客观性。

![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20241220112127.png)

当开始为优化收集数据时，对于训练集，一般建议至少有30个例子（示例），但是如果可以的话，最好达到300个示例。一些优化器只需要`trainset`；另一些则需要同时提供`trainset`和`valset`。针对提示优化（prompt optimizers）而言，建议你将数据进行一个与常规深度学习截然相反的切分比例：**20%数据用于训练，80%用于验证**。

 *为什么提示优化器要用 20% 训练，80% 验证，而深度神经网络通常相反？*
- 这是因为提示优化通常数据量较小，需要更多的数据来评估模型的泛化能力，防止过拟合到少量训练数据上。深度神经网络通常有大量数据，过拟合风险相对较小，因此可以将更多数据用于训练。
- 提示优化关注的是如何构建有效的提示，而不是像深度神经网络那样学习复杂的特征表示。验证集上的表现更能反映提示的质量。



在进行了最初的几轮优化后，你可能会对结果非常满意，或者取得了很大进展但对最终程序或指标的某些方面不满意。这时，你需要回到第一步（在 DSPy 中编程）并重新审视主要问题。你的任务定义是否清晰？是否需要收集（或在线查找）更多数据？是否需要更新指标？是否需要使用更复杂的优化器？是否需要考虑像 DSPy 断言这样的高级功能？或者，也许最重要的是，是否需要在 DSPy 程序本身中添加更多复杂性或步骤？是否需要按顺序使用多个优化器？


迭代开发是关键。DSPy 提供了逐步迭代的工具：你可以迭代数据集本身（不断改进数据），迭代程序结构（拆解步骤，增加模块），迭代对断言与度量的定义，以及迭代你的优化流程来优化复杂的 LM 程序。

# DSPy 优化器（以前称为 Teleprompters）

**DSPy 优化器**是一种算法，它可以优化提示（prompts）和/或LM权重，从而最大化你定义的评价指标（metrics），例如准确率。

即**优化器的核心作用是自动调整程序参数，以提高程序性能。**

一个典型的 DSPy 优化器需要三个要素：

- **你的 DSPy 程序**。这可能是一个单个模块（例如 `dspy.Predict`）或一个复杂的多模块程序例如一个RAG问答系统、一个Agent等。
- **你的评价指标（metric）**。这是一个评估程序输出并为其分配分数（越高越好）的函数。（指标定义了你希望程序达到的目标，例如准确率、召回率、F1 分数等。优化器会根据指标来调整程序参数，使其性能更好）
- **一些训练输入**。这可能非常小（即只有 5 或 10 个示例）且不完整（只有程序的输入，没有任何标签）。
    -  为什么可以只有输入，没有标签？
        - 一些优化器（例如 `BootstrapFewShot`）可以利用无标签数据进行自举学习，生成伪标签来辅助优化。

如果你碰巧有很多数据，DSPy 可以利用这些数据。但你也可以从小规模开始并获得很好的结果。

**注意：DSPy优化器曾被称为teleprompters，现在正式更名为optimizers。这一变化会在库和文档中逐步体现。

## DSPy 优化器调整什么？它是如何调整的？

DSPy 中的不同优化器将通过以下方式调整程序的质量：**自动合成高质量的few-shot示例**到每个模块的提示中，例如 `dspy.BootstrapRS`[1](https://arxiv.org/abs/2310.03714)；为每个提示提出并智能地探索更好的自然语言指令（instructions），例如 `dspy.MIPROv2`[2](https://arxiv.org/abs/2406.11695)；**你的模块构建数据集并利用这些数据对底层语言模型（LM）权重进行微调**，例如 `dspy.BootstrapFinetune`[3](https://arxiv.org/abs/2407.10930)。

即三种主要的优化策略：
- **合成少样本示例**: 通过生成一些高质量的示例来构建提示，帮助模型更好地理解任务。
- **优化自然语言指令**: 调整提示中的指令，使其更清晰、更准确地指导模型。
- **微调 LM 权重**: 使用特定任务的数据来微调模型的参数，使其在该任务上表现更好。


优化器的组合使用可以进一步提高性能，这也是 DSPy 的一个重要优势。

## 当前有哪些可用的 DSPy 优化器？

可以通过 `from dspy.teleprompt import *` 访问优化器。

### 自动少样本Few-Shot学习

这些优化器会自动生成few-shot示例（即在提示中提供的演示样例）并将其加入提示中，从而实现few-shot学习。。

1. **`LabeledFewShot`**: 简单地从提供的带标签的输入和输出数据点构建少样本示例（演示）。需要参数 `k`（提示的示例数量）和 `trainset` 以从中随机选择 `k` 个示例。
    - *这个优化器做了什么？
        - 别被吓到，它只是简单地从训练集中选择一些示例，并将它们添加到提示中，作为模型的演示。
2. **`BootstrapFewShot`**: 使用一个`teacher`模块（默认为你的程序本身）生成完整的演示案例以及训练集中的标注示例。该过程会利用metric来验证示例质量，只保留通过检查的示例。可调节参数包括`max_labeled_demos`（从训练集中选的演示示例数量）和`max_bootstrapped_demos`（由teacher程序自动生成的额外示例数量）。
    - 这里的“自举”到底是什么意思？
        - “自举”指的是使用模型本身来生成新的示例。`teacher` 模块会根据已有的示例生成新的示例，并使用指标来评估这些示例的质量。
3. **`BootstrapFewShotWithRandomSearch`**: 应用 `BootstrapFewShot` 多次，并在生成的演示中进行随机搜索，并选择优化后的最佳程序。参数与 `BootstrapFewShot` 的参数相同，并添加了 `num_candidate_programs`，它指定了在优化过程中评估的随机程序的数量，包括未编译程序的候选程序、`LabeledFewShot` 优化程序、使用未打乱示例的 `BootstrapFewShot` 编译程序以及使用随机示例集的 `BootstrapFewShot` 编译程序的 `num_candidate_programs`。
    -  为什么要进行随机搜索？
        - 随机搜索可以帮助找到更好的演示组合，因为 `BootstrapFewShot` 生成的演示可能不是最优的。
4. **`KNNFewShot`**: 使用 k 最近邻算法为给定的输入示例查找最近的训练示例演示。然后，这些最近邻演示被用作 BootstrapFewShot 优化过程的训练集。
    -  什么是 k 最近邻算法？
        - k 最近邻算法是一种基于实例的学习方法，它根据输入示例与训练集中示例的相似度来选择演示。

### 自动prompt优化

这些优化器为提示生成最佳指令，在 MIPROv2 的情况下，还可以优化少样本演示集。

1. [**`COPRO`**](https://dspy.ai/deep-dive/optimizers/copro): 为每个步骤生成和改进新指令，并使用坐标上升（使用指标函数和 `trainset` 进行爬山）对其进行优化。参数包括 `depth`，它是优化器运行的提示改进迭代次数。
    - 什么是坐标上升？
        - 坐标上升是一种优化算法，它沿着每个坐标轴依次进行优化，直到收敛。
2. [**`MIPROv2`**](https://dspy.ai/deep-dive/optimizers/miprov2): 在每个步骤中生成prompt和few-shot示例。指令生成是数据感知和演示感知的。使用贝叶斯优化在你的模块中有效地搜索生成prompt/demo的空间。
    - 什么是贝叶斯优化？
        - 贝叶斯优化是一种用于优化黑盒函数的全局优化算法，它使用概率模型来指导搜索过程。



## 我应该使用哪个优化器？

最终，找到“正确”的优化器和适合你任务的最佳配置需要进行实验。在 DSPy 中取得成功仍然是一个迭代过程 - 在你的任务上获得最佳性能需要你进行探索和迭代。

话虽如此，以下是入门的一般指导：

- 如果你只有**很少的示例**（大约 10 个），请从 `BootstrapFewShot` 开始。
- 如果你有**更多的数据**（50 个示例或更多），请尝试 `BootstrapFewShotWithRandomSearch`。
- 如果你只想进行**指令优化**（即你想保持提示为 0 样本），请使用 [配置为 0 样本优化的 `MIPROv2`](https://dspy.ai/deep-dive/optimizers/miprov2#optimizing-instructions-only-with-miprov2-0-shot)。
- 如果你愿意使用更多的推理调用来执行**更长的优化运行**（例如 40 次试验或更多），并且有足够的数据（例如 200 个示例或更多以防止过拟合），那么请尝试 `MIPROv2`。






### 第一步：导入必要的库和模块

`Evaluate` 是一个评估工具，它能够根据指定的指标（这里是 `gsm8k_metric`）来评估模型的性能，告诉我们模型回答问题的准确率。

`GSM8K` 是一个常用的数学推理数据集，用于测试和训练模型的数学推理能力。通过在这个数据集上训练和评估模型，我们可以了解模型的推理能力。


In [1]:
# %load_ext autoreload
# %autoreload 2
# import sys; sys.path.append('/future/u/okhattab/repos/public/stanfordnlp/dspy')

import dspy
from dspy.evaluate import Evaluate
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

### 第二步：加载数据集和配置语言模型

准备数据和设置语言模型。


In [33]:
#加载了 GSM8K 数据集。
gms8k = GSM8K()


100%|██████████| 1319/1319 [00:00<00:00, 41219.89it/s]


In [3]:

import dspy
#换成自己的模型
turbo = dspy.LM('ollama_chat/llama3.2:1b', api_base='http://localhost:11434', api_key='')
# lm = dspy.LM('ollama_chat/llama3.2:1b', api_base='http://localhost:11434', api_key='')
#将数据集分为训练集和开发集。
trainset, devset = gms8k.train, gms8k.dev
# 将 DSPy 的全局语言模型设置为我们初始化的 `1b 模型。
dspy.settings.configure(lm=turbo)


*   **为什么要分训练集和开发集？** 训练集用于训练模型或优化提示，而开发集用于评估优化后的模型性能。这样可以确保我们是在一个未见过的数据集上评估模型的泛化能力，从而得到一个更可靠的性能估计。同时还有个可选的验证集，如BootstrapFewShotWithRandomSearch优化器会在训练集上训练时在验证集上评估，选择在验证集上表现最好的参数配置作为最后的优化结果。

![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20241220112127.png)

验证下模型的连接

In [34]:
response1 = turbo("看这里有只猫，", temperature=0.7) 
response1

['是的，这里可能有只猫。虽然我不在这里，但如果你想要知道更多关于猫的信息或需要一些关于猫的资源，那么我可以帮助你了。']

In [35]:
trainset[:1]

[Example({'question': "The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?", 'gold_reasoning': "Ella's score is 40 items - 4 items = <<40-4=36>>36 items. Half of Ella's score is 36 items / 2 = <<36/2=18>>18 items. So, Marion's score is 18 items + 6 items = <<18+6=24>>24 items.", 'answer': '24'}) (input_keys={'question'})]



### 第三步：设置评估器

*   ** 提问思考：** 如何使用自己定义的指标进行评估？

    *   可以使用一个简单的 Python 循环来遍历开发集，运行程序，并使用指标函数计算每个样本的分数。


In [ ]:
scores = []
for x in devset:
    pred = program(**x.inputs())
    score = metric(x, pred)
    scores.append(score)

遍历开发集中的每个样本 `x`，使用 `x` 的输入运行程序 `program`，得到预测结果 `pred`，然后使用指标函数 `metric` 计算 `pred` 相对于 `x` 的分数，并将分数添加到 `scores` 列表中。

DSPy 提供了 `Evaluate` 工具类来简化评估过程：

In [ ]:
from dspy.evaluate import Evaluate

# 设置评估器，可以在代码中重复使用。
evaluator = Evaluate(devset=你的开发集, num_threads=1, display_progress=True, display_table=5)

# 启动评估。
evaluator(你的程序, metric=你的指标)

*   这里的`display_progress=True`会显示评估过程进度，`display_table=5`则会展示示例输出的表格情况（例如显示前5个样本的输入、输出和得分）。

* ** 提问思考：** `Evaluate` 工具类提供了哪些功能？

    *   `Evaluate` 工具类提供了并行评估、显示进度条、显示样本输入/输出以及指标分数等功能。

应用：


In [39]:
NUM_THREADS = 4
#创建了一个评估器实例。
evaluate = Evaluate(devset=devset[:20], metric=gsm8k_metric, num_threads=NUM_THREADS, display_progress=True, display_table=10,provide_traceback=True)
# devset=devset[:] 指定了用于评估的数据集
# metric=gsm8k_metric 指定了评估指标，即判断答案是否正确的函数。
# num_threads=NUM_THREADS 设置了评估线程数。
# display_progress=True 显示评估进度条。
# display_table=0 不显示详细的评估结果表格。

*   **这里我们使用了多线程评估？** 多线程评估可以利用多核 CPU 的计算能力，并行处理多个样本的评估，从而显著加快评估速度，特别是当数据集很大时。
*   **`gsm8k_metric` 具体是怎么判断答案是否正确的？** `gsm8k_metric` 函数会比较模型生成的答案和真实答案，通常是通过比较最终的数值结果。如果模型答案的数值与真实答案的数值相同（或在一定的误差范围内），则认为答案正确。


**接下来我们开始用最原始的Predict方法去问**
注意有报错是说明没有提取到我们规定的想要的字段，即llama3.2:1b模型能力不足output_fields规定的['answer']字段，是正常现象，也是我们优化的目标。


In [37]:
normal=dspy.Predict("question -> answer")

evaluate(normal)

Average Metric: 0.00 / 2 (0.0%):   5%|▌         | 1/20 [00:00<00:00, 83.37it/s]

2024/12/20 19:00:46 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'Vanessa wants to buy a dress she saw at the mall, which costs $80, and she already has $20 in savings. Her parents give her $30 every week, but she also spends $10 each weekend at the arcades. How many weeks will she have to wait until she can gather enough money to buy the dress?', 'gold_reasoning': 'Vanessa needs $80 – $20 = $<<80-20=60>>60 to buy the dress. She manages to gather $30 - $10 = $<<30-10=20>>20 each week The number of weeks she has to wait is 60 ÷ 20 = <<60/20=3>>3 weeks.', 'answer': '3'}) (input_keys={'question'}): 'str' object has no attribute 'items'
Stack trace:
Traceback (most recent call last):
  File "D:\Program\anaconda3\Lib\site-packages\dspy\adapters\base.py", line 25, in __call__
    value = self.parse(signature, output, _parse_values=_parse_values)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Program\anaconda3\Lib\site-packages\d

Average Metric: 1.00 / 3 (33.3%):  10%|█         | 2/20 [00:00<00:00, 117.69it/s]

2024/12/20 19:00:46 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'Trey is raising money for a new bike that costs $112. He plans to spend the next two weeks selling bracelets for $1 each. On average, how many bracelets does he need to sell each day?', 'gold_reasoning': 'He needs to sell 112 bracelets because 112 divided by 1 equals <<112/1=112>>112. He has 14 days to sell bracelets because there are seven days in a week and he has two weeks seven times 2 equals <<7*2=14>>14. He has to sell 8 bracelets a day because 112 divided by 14 equals <<112/14=8>>8.', 'answer': '8'}) (input_keys={'question'}): 'str' object has no attribute 'items'
Stack trace:
Traceback (most recent call last):
  File "D:\Program\anaconda3\Lib\site-packages\dspy\adapters\base.py", line 25, in __call__
    value = self.parse(signature, output, _parse_values=_parse_values)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Program\anaconda3\Lib\site-package

Average Metric: 3.00 / 5 (60.0%):  20%|██        | 4/20 [00:00<00:00, 142.87it/s]

2024/12/20 19:00:46 ERROR dspy.utils.parallelizer: Error processing item Example({'question': "Erika and her 3 siblings are drawing with chalk outside. Another 3 friends join them and ask if they can help with the drawing. Erika loses 2 pieces of chalk as they are counting and the group realizes there isn't enough chalk for everyone. Erika’s mom brings out another 12 pieces of chalk for them to share and there is now enough chalk for everyone to have 3 pieces each. How many pieces of chalk did Erika and her siblings originally have?", 'gold_reasoning': 'Altogether, there are 1 Erika + 3 siblings + 3 friends = <<1+3+3=7>>7 people drawing outside. The group has a total of 7 people * 3 pieces of chalk each = <<7*3=21>>21 pieces of chalk altogether. Before Erika’s mom helped, they had 21 total pieces of chalk - 12 pieces of chalk Erika’s mom brought = <<21-12=9>>9 pieces of chalk. Then adding the chalk that Erika lost shows that the group originally had 9 pieces of chalk + 2 lost pieces of

Average Metric: 4.00 / 17 (23.5%): 100%|██████████| 20/20 [00:00<00:00, 434.82it/s]

2024/12/20 19:00:46 INFO dspy.evaluate.evaluate: Average Metric: 4.0 / 20 (20.0%)


,question,gold_reasoning,example_answer,pred_answer,gsm8k_metric,answer
0,"20 birds migrate on a seasonal basis from one lake to another, sea...",The birds' flying distance between Lake Jim through lake Disney to...,2200,{2200},✔️ [True],NaN
1,"Wendy went to the dentist for a cleaning, two fillings, and a toot...",Wendy’s dentist bill was 5 * $120 = $<<5*120=600>>600. She got two...,290,$840,,NaN
2,Karen is packing her backpack for a long-distance hike. She packs ...,First find the weight of food Karen eats every hour: 2 pounds * 1/...,34,{weight = (20 + 10 + 20) * 6 - (2 + (1/3)*10)},,NaN
3,Rita is reading a five-chapter book with 95 pages. Each chapter ha...,Let P be the number of pages in the first chapter. The second chap...,13,Let's denote the number of pages in the first chapter as x. Since ...,,NaN
4,Benjamin collects 6 dozen eggs a day. Carla collects 3 times the n...,Carla:3(6)=18 dozen eggs Trisha: 6-4=<<6-4=2>>2 dozen eggs Total: ...,26,{26},✔️ [True],NaN


20.0

最后得分是20分,即用llama3.2:1b模型回答这些数学题只有20%的正确率

### 第五步：优化 定义思维链 (CoT) 模块

接下来我们用之前学过的COT思维链优化，看看成功率到底有没有提升

这一步定义了一个 DSPy 模块，该模块使用思维链提示策略来解决问题。

*   `class CoT(dspy.Module):` 定义了一个名为 `CoT` 的类，它继承自 `dspy.Module`。在 DSPy 中，`Module` 是构建复杂提示策略的基本单元。
    *   `self.prog = dspy.ChainOfThought("question -> answer")` 初始化了一个思维链程序。`"question -> answer"` 是一个签名，它告诉 DSPy 输入是一个问题，输出是一个答案。DSPy 会根据这个签名来自动生成prompt。
*   `def forward(self, question):` 前向传播方法，定义了如何使用这个模块。
    *   `return self.prog(question=question)` 将输入的问题传递给思维链程序，并返回结果。

In [24]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("question -> answer")
    
    def forward(self, question):
        return self.prog(question=question)

In [25]:
# 创建CoT模块实例
cot = CoT()

这一步使用评估器评估编译后的思维链模型的性能。

*   `evaluate(cot_bs,)` 调用评估器的 `__call__` 方法，评估 `cot` 模型在整个开发集上的性能。评估结果将打印到控制台。

*   **评估结果是什么样的？** 评估结果通常会显示模型在开发集上的准确率，即模型正确回答的问题比例。例如，输出可能类似于 `25.0`, 表示模型在25%的问题上给出了正确的答案。
*   **如何解读评估结果？** 评估结果可以帮助我们了解模型的性能。如果准确率较高，说明模型的推理能力较强；如果准确率较低，说明模型还需要进一步改进。

注意有报错是说明没有提取到我们规定的想要的字段，即llama3.2:1b模型能力不足没有按我们的规定生成COT链路output_fields规定的['reasoning', 'answer']字段，是正常现象，这也是后续可以优化的地方。

In [40]:
# 使用evaluate进行评估
evaluate(cot)

Average Metric: 2.00 / 7 (28.6%):  30%|███       | 6/20 [00:00<00:00, 428.69it/s]

2024/12/20 19:07:23 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'Vanessa wants to buy a dress she saw at the mall, which costs $80, and she already has $20 in savings. Her parents give her $30 every week, but she also spends $10 each weekend at the arcades. How many weeks will she have to wait until she can gather enough money to buy the dress?', 'gold_reasoning': 'Vanessa needs $80 – $20 = $<<80-20=60>>60 to buy the dress. She manages to gather $30 - $10 = $<<30-10=20>>20 each week The number of weeks she has to wait is 60 ÷ 20 = <<60/20=3>>3 weeks.', 'answer': '3'}) (input_keys={'question'}): Expected dict_keys(['reasoning', 'answer']) but got dict_keys([])
Stack trace:
Traceback (most recent call last):
  File "D:\Program\anaconda3\Lib\site-packages\dspy\adapters\base.py", line 25, in __call__
    value = self.parse(signature, output, _parse_values=_parse_values)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Program\a

Average Metric: 2.00 / 8 (25.0%):  35%|███▌      | 7/20 [00:00<00:00, 291.75it/s]

2024/12/20 19:07:23 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'Trey is raising money for a new bike that costs $112. He plans to spend the next two weeks selling bracelets for $1 each. On average, how many bracelets does he need to sell each day?', 'gold_reasoning': 'He needs to sell 112 bracelets because 112 divided by 1 equals <<112/1=112>>112. He has 14 days to sell bracelets because there are seven days in a week and he has two weeks seven times 2 equals <<7*2=14>>14. He has to sell 8 bracelets a day because 112 divided by 14 equals <<112/14=8>>8.', 'answer': '8'}) (input_keys={'question'}): 'str' object has no attribute 'items'
Stack trace:
Traceback (most recent call last):
  File "D:\Program\anaconda3\Lib\site-packages\dspy\adapters\base.py", line 25, in __call__
    value = self.parse(signature, output, _parse_values=_parse_values)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Program\anaconda3\Lib\site-package

Average Metric: 3.00 / 10 (30.0%):  45%|████▌     | 9/20 [00:00<00:00, 225.04it/s]

...

2024/12/20 19:07:23 ERROR dspy.utils.parallelizer: Error processing item Example({'question': "Erika and her 3 siblings are drawing with chalk outside. Another 3 friends join them and ask if they can help with the drawing. Erika loses 2 pieces of chalk as they are counting and the group realizes there isn't enough chalk for everyone. Erika’s mom brings out another 12 pieces of chalk for them to share and there is now enough chalk for everyone to have 3 pieces each. How many pieces of chalk did Erika and her siblings originally have?", 'gold_reasoning': 'Altogether, there are 1 Erika + 3 siblings + 3 friends = <<1+3+3=7>>7 people drawing outside. The group has a total of 7 people * 3 pieces of chalk each = <<7*3=21>>21 pieces of chalk altogether. Before Erika’s mom helped, they had 21 total pieces of chalk - 12 pieces of chalk Erika’s mom brought = <<21-12=9>>9 pieces of chalk. Then adding the chalk that Erika lost shows that the group originally had 9 pieces of chalk + 2 lost pieces of

Average Metric: 5.00 / 16 (31.2%): 100%|██████████| 20/20 [00:00<00:00, 344.86it/s]

2024/12/20 19:07:23 INFO dspy.evaluate.evaluate: Average Metric: 5.0 / 20 (25.0%)


,question,gold_reasoning,example_answer,reasoning,pred_answer,gsm8k_metric,answer
0,"20 birds migrate on a seasonal basis from one lake to another, sea...",The birds' flying distance between Lake Jim through lake Disney to...,2200,The problem involves calculating the total distance traveled by a ...,2000 miles,,NaN
1,"Wendy went to the dentist for a cleaning, two fillings, and a toot...",Wendy’s dentist bill was 5 * $120 = $<<5*120=600>>600. She got two...,290,"To find out how much Wendy paid for the tooth extraction, we need ...","Wendy paid for two fillings and a tooth extraction, which costs $1...",,NaN
2,Karen is packing her backpack for a long-distance hike. She packs ...,First find the weight of food Karen eats every hour: 2 pounds * 1/...,34,"To calculate the weight Karen is carrying after six hours, we need...",Karen is carrying 3 pounds of gear after six hours.,,NaN
3,Rita is reading a five-chapter book with 95 pages. Each chapter ha...,Let P be the number of pages in the first chapter. The second chap...,13,"Let's assume the first chapter has x pages. Then, the second chapt...",The first chapter of the book has 19 pages.,,NaN
4,Benjamin collects 6 dozen eggs a day. Carla collects 3 times the n...,Carla:3(6)=18 dozen eggs Trisha: 6-4=<<6-4=2>>2 dozen eggs Total: ...,26,"To find Carla's collection, multiply Benjamin's by 3: 6 * 3 = 18. ...",Benjamin collects 6 dozen eggs a day. Carla collects 18 dozen eggs...,,NaN
5,Roy spends 2 hours on sports activities in school every day. He go...,He goes to.school 5 days a week so if he misses 2 days within the ...,6,To find the number of hours Roy spent on sports activities that we...,4,,NaN
6,Cameron is printing her thesis in the school library and has 400 A...,"If she had 400 papers, the total number of documents she separated...",240,"To find the number of invalid papers, multiply the total number of...",The number of valid documents is the total number of papers minus ...,✔️ [True],NaN
7,Burt spent $2.00 on a packet of basil seeds and $8.00 on potting s...,He spent $2.00 on seeds and $8.00 on soil for a total of 2+8 = $<<...,90,The total cost of the basil seeds and potting soil was $10.00. Bur...,$90.00,✔️ [True],NaN
8,Martha's cat catches 3 rats and 7 birds. Cara's cat catches 3 less...,First find the total number of animals Martha's cat catches: 3 rat...,47,"Cara's cat catches five times as many animals as Martha's cat, plu...",Martha's cat catches 3 rats and 7 birds. Cara's cat catches 47 ani...,✔️ [True],NaN
9,Trey is raising money for a new bike that costs $112. He plans to ...,He needs to sell 112 bracelets because 112 divided by 1 equals <<1...,NaN,NaN,NaN,,8


25.0

In [41]:
dspy.inspect_history(n=1)





[2024-12-20T19:07:23.914804]

System message:

Your input fields are:
1. `question` (str)

Your output fields are:
1. `reasoning` (str)
2. `answer` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

Inputs will have the following structure:

[[ ## question ## ]]
{question}

Outputs will be a JSON object with the following fields.

{
  "reasoning": "{reasoning}",
  "answer": "{answer}"
}

In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `answer`.


User message:

[[ ## question ## ]]
Bob, Tom, Sally, and Jerry had dinner at their favorite pizzeria. They decide to share 2 pizzas. Bob ate half of a pizza on his own. Tom ate one-third of a pizza. Sally wasn't very hungry and only ate one-sixth of a pizza, and Jerry ate a quarter of a pizza. If each pizza is cut into 12 slices, how many slices were left over?

Respond with a JSON object in the following order of fields: `reaso

最后的正确率是25%，相比较不使用COT提升了5%（在20道题的测试上）

### 优化器 `LabeledFewShot` 

使用你想要的 `k` 值（即每个预测器使用的示例数量）来实例化 `LabeledFewShot`。

`LabeledFewShot` 类实现了一种少样本学习策略，它通过为模型提供少量带标签的示例来帮助模型进行预测。`compile` 方法是该类的核心，它负责选择示例并将它们分配给 `student` 模型中的各个预测器。代码中使用了随机采样或选择前 `k` 个示例的方式，并考虑了训练集为空的情况。注释部分提供了一些关于代码行为和潜在交互的信息。

In [42]:
from dspy.teleprompt import LabeledFewShot
k = 2  # 你可以根据需要调整这个值
fewshot = LabeledFewShot(k=k)

In [43]:
compiled_turbo_few_shot = fewshot.compile(cot.deepcopy(), trainset=trainset[:20])

**这意味着，在编译阶段 (`fewshot.compile(...)`)，`LabeledFewShot` 策略会在这 20 个训练示例中进行操作。具体来说，它会针对 `cot` 模型中的每一个 predictor 执行以下操作：**

*   因为 `LabeledFewShot` 的 `compile` 方法默认使用随机采样 (`sample=True`，参考之前的代码解释)，所以它将从这 20 个示例中 **随机** 选取 2 个 (`k=2`) 作为该 predictor 的 `demos`。

**这个有优化器实际的作用是每次从这 20 个实例中随机选择 2 个作为 few-shot 示例。**

In [44]:
for predictor in compiled_turbo_few_shot.predictors():
    print(f"Predictor: {predictor}")
    print(f"  Demos (length: {len(predictor.demos)}):")
    for demo in predictor.demos:
        print(f"    - Question: {demo.question}")
        print(f"    - Answer: {demo.answer}") # Assuming your demos have 'question' and 'answer' fields.
    print("-----")

Predictor: Predict(StringSignature(question -> reasoning, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))
  Demos (length: 2):
    - Question: A tank contains 6000 liters of water, 2000 liters evaporated, and then 3500 liters were drained by Bob. How many liters are in the tank if it now rains for 30 minutes and every 10 minutes 350 liters of rain are added to the tank?
    - Answer: 1550
    - Question: John takes 3 days off of streaming per week.  On the days he does stream, he st

In [45]:
evaluate(compiled_turbo_few_shot)

Average Metric: 5.00 / 20 (25.0%): 100%|██████████| 20/20 [01:47<00:00,  5.36s/it]

2024/12/20 19:15:44 INFO dspy.evaluate.evaluate: Average Metric: 5 / 20 (25.0%)


,question,gold_reasoning,example_answer,reasoning,pred_answer,gsm8k_metric
0,"20 birds migrate on a seasonal basis from one lake to another, sea...",The birds' flying distance between Lake Jim through lake Disney to...,2200,The problem involves calculating the total distance traveled by mu...,950,
1,"Wendy went to the dentist for a cleaning, two fillings, and a toot...",Wendy’s dentist bill was 5 * $120 = $<<5*120=600>>600. She got two...,290,Wendy paid $240 for her dentist bill.,$300,
2,Karen is packing her backpack for a long-distance hike. She packs ...,First find the weight of food Karen eats every hour: 2 pounds * 1/...,34,Not supplied for this particular example.,180.0,
3,Rita is reading a five-chapter book with 95 pages. Each chapter ha...,Let P be the number of pages in the first chapter. The second chap...,13,Let's denote the number of pages in the first chapter as x. Since ...,19,
4,Benjamin collects 6 dozen eggs a day. Carla collects 3 times the n...,Carla:3(6)=18 dozen eggs Trisha: 6-4=<<6-4=2>>2 dozen eggs Total: ...,26,Not supplied for this particular example.,72,
5,Roy spends 2 hours on sports activities in school every day. He go...,He goes to.school 5 days a week so if he misses 2 days within the ...,6,Not supplied for this particular example.,40,
6,Cameron is printing her thesis in the school library and has 400 A...,"If she had 400 papers, the total number of documents she separated...",240,"To find the number of invalid papers, multiply 400 by 40% (or 0.4)...",240,✔️ [True]
7,Burt spent $2.00 on a packet of basil seeds and $8.00 on potting s...,He spent $2.00 on seeds and $8.00 on soil for a total of 2+8 = $<<...,90,"To find the net profit, first calculate the total cost of seeds an...",$90.00,✔️ [True]
8,Martha's cat catches 3 rats and 7 birds. Cara's cat catches 3 less...,First find the total number of animals Martha's cat catches: 3 rat...,47,Martha's cat catches 3 + (5 * 7) = 28 animals.\n\nCara's cat catch...,7,
9,Trey is raising money for a new bike that costs $112. He plans to ...,He needs to sell 112 bracelets because 112 divided by 1 equals <<1...,8,"To find out how many bracelets Trey needs to sell per day, we firs...",$224,


25.0

可以看到答案的格式像例子一样更简洁甚至只有一个数字了，这也指导reasoning的思考是向着最后的数字结果去的

又进步了一些！

In [46]:
dspy.inspect_history(1)





[2024-12-20T19:15:44.684938]

System message:

Your input fields are:
1. `question` (str)

Your output fields are:
1. `reasoning` (str)
2. `answer` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `answer`.


User message:

This is an example of the task, though some input or output fields are not supplied.

[[ ## question ## ]]
A tank contains 6000 liters of water, 2000 liters evaporated, and then 3500 liters were drained by Bob. How many liters are in the tank if it now rains for 30 minutes and every 10 minutes 350 liters of rain are added to the tank?

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## answer ## ]]`, and then ending with the mark

### 优化器：BootstrapFewShot

前置概念复习：

**演示 (Demo / Demonstration):**  在 DSPy 中，一个演示是一个关于如何执行特定任务的单一示例。它本质上代表了一个输入-输出对，展示了对于特定输入，语言模型（LM）应该表现出的理想行为。你可以把它想象成给学生展示的一个解题示例。一个演示是 `dspy.Example` 类的一个实例。它包含了一个模块（预测器）的输入、相应的输出，以及可能在过程中生成的中间步骤或解释。
    *   **示例:** 对于一个问答任务，一个演示可能长这样：
        ```
        输入 (问题): 法国的首都是什么？
        输出 (答案): 巴黎
        ```
    *   **关键思想:** Demo至关重要，因为它们通过展示具体的例子来指导语言模型，告诉它应该如何处理输入并生成输出。
*   **预测器 (Predictor):** 在 DSPy 中，`Predictor` 是一个核心组件，它封装了语言模型（LM）并处理与它的交互。它定义了如何提示语言模型以及如何处理它的输出。可以把它看作是一个特定的指令或模块，它使用语言模型来执行更大程序中的一个子任务。例如 `dspy.Predict`、`dspy.ChainOfThought` 以及 DSPy 中的其他模块。
*   **签名 (Signature):** 在 `dspy` 的语境中，签名定义了一个模块的输入和输出字段。你可以把它想象成一个合约，规定了一个模块接受什么类型的数据以及产生什么类型的数据。这对于确保不同的模块可以连接并无缝地协同工作至关重要。
*   **轨迹 (Trace):** 轨迹捕获了 DSPy 程序的执行流程。它记录了所采取的步骤序列，包括使用的预测器、它们的输入和输出。它就像一个详细的日志，记录了程序单次运行期间发生的事情，对于调试和理解程序如何得出特定输出至关重要。
*   **教师模型和学生模型 (Teacher and Student Models):** 在这个上下文中，教师和学生都是语言模型。**教师**模型用于生成演示（解题示例）。**学生**模型是使用教师生成的演示进行优化（微调）的模型。
*   **标注数据 vs. 未标注数据 (Labeled vs. Unlabeled Data):**
    *   **标注数据** 带有预定义的正确答案（就像一个问题及其对应的答案）。（我们的数据集市标注答案）
    *   **未标注数据** 只有输入（就像一个没有答案的问题）。
    *   `BootstrapFewShot` 的优势是利用未标注数据来增强有限的标注数据，并提高学生模型的性能。

**训练前和训练后的差异**

*   **训练前:** 学生模型对任务有一个基本的理解，可能只有少量甚至没有演示。它在未见过的例子上的泛化能力和表现是有限的。
*   **训练后:** 在使用 `BootstrapFewShot` 训练后，学生模型接触到了更丰富的演示集合，包括标注示例和教师生成的自举示例。这使得模型的性能得到提升，泛化能力更强。学生模型的提示中会包含更丰富的 `demos` 集合，从而带来更好的性能。

**自举迭代生成演示的流程详解**

想象一下，你是一位老师 (teacher)，你想教一个学生 (student) 解决数学应用题。你手里有一堆题目 (trainset)，这些示例中有些可能有答案（标注），但许多可能没有（未标注）。。你的目标是教会学生解题的思路和步骤，而不仅仅是答案。为了达到这个目的，你创建了一套教学方法 (BootstrapFewShot)，这套方法可以概括为以下几个步骤，注意**整个过程没有真正的学生参与，是教师模型自己在“自问自答”**：

**1. 初始化： 做好准备工作**

- 你首先清点了一下手头的资源，设置了一些规则：
    - **最多使用多少个自举的例题 (max_bootstraps_demos):**  你不可能无限地讲解例题，所以需要设定一个上限，即最大自举演示数量。。
    - **每个题目最多尝试讲解多少遍 (max_rounds):**  教师模型为每个示例尝试生成成功演示的最大次数。 这是因为教师模型在解题时，不一定每次都能得到正确答案或者说能生成一个高质量的演示。通过设置 max_rounds，可以让教师模型在一个题目上尝试多次，增加了成功的机会。
    - **记录讲解成功的题目:**  你需要一个笔记本 (`bootstrapped`) 来记录哪些题目你已经成功地演示了解题步骤。
    - **记录每个教学步骤的解题过程 (name2traces):** 你还有一个笔记本，用来记录你在讲解每个题目时，每一步是怎么思考的，用了哪些方法 (predictor)，以及这个步骤的输入和输出。我们把每一步的思考过程称为一个“轨迹”(trace)，这个笔记本就用来收集这些轨迹。

**2. 遍历训练集: 挑选题目**

- 你开始逐一挑选题目 (example) 给学生讲解。你手里有一本题目集 (trainset)，你需要从头到尾，一道题一道题地过一遍。即该流程遍历 `trainset` 中的每个示例。

**3. 多轮迭代: 反复讲解**

- 对于每个示例，教师模型会尝试生成一个演示，最多尝试 `max_rounds` 次。这种迭代方法至关重要，因为教师可能不会总是在第一次尝试时就产生高质量的演示。

**4. 准备预测器: 为教师设置舞台准备讲解**

- 在每一轮讲解之前，你需要做一些准备工作：
    - **将预测器的 `demos` 属性缓存起来:**  `demos` 就像是你之前给学生总结的解题技巧和例题。在讲解这道题之前, 你会先把这些技巧和例题暂时放到一边 (缓存起来). 为什么呢？因为你现在要教学生的是这道题的解题思路，而不是让他直接套用之前的例题。
    - **并从 `demos` 中移除当前示例:**  你还要确保你选择的例题里面没有当前这道题，不然就成了“循环论证”了。这就好比你在讲解一道题时，不会把这道题本身当作例题来讲。

**5. 进行预测: 教师模型生成解决方案 (老师自己尝试解答)**

-   **“现在你开始正式给学生讲解这道题了 (使用教师模型对当前示例进行预测)。”**  这里是**教师模型自己尝试解答**的部分。教师模型会接收题目（示例的输入），然后根据自己当前的“理解”（模型参数和 `demos`）尝试解答这道题。
-   **“你一边讲解，一边在另一个笔记本上记录下你的每一个思考步骤, 也就是所谓的"轨迹" (trace)”。**  这个“笔记本”就是 `dspy` 用来记录推理轨迹的地方。教师模型并不是真的在“讲解”，而是**在进行内部的推理计算**。每一步的推理都会被记录下来，包括：
    -   `你用了什么方法/思路 (predictor)`：使用了哪个模块进行推理。
    -   `这个步骤的已知条件是什么 (inputs)`：输入给这个模块的数据是什么。
    -   `你得出了什么中间结论 (outputs)`：这个模块输出的结果是什么。

**6. 评估预测结果: 检查答案 (评估老师的结果判断讲解是否有效)**

-   **“你讲解完一种解法后，需要检查教师模型自己生成的预测结果 (prediction)是否正确。”** 
-   **“如果你有一个评估标准 (metric): 你会根据这个标准来判断学生的解答是否正确，或者是否足够好 (metric_threshold)。”**  这一步是判断讲解是否有效的关键。
    -   **有 `metric` 的情况:** `metric` 函数将教师的输出与预期输出（如果有）进行比较，或根据某些标准评估其质量。例如在 GSM8K 数据集中，评估标准会比较模型生成的答案和标准答案是否一致，则会使用这个标准来评估预测结果的质量。
    -   **没有 `metric` 的情况:** 默认情况下，生成的输出被认为是可接受的。
    -   **阈值判断:** 如果设置了 `metric_threshold`，则将 `metric` 的输出与此阈值进行比较。如果指标的值达到或超过阈值，则认为生成是成功的。
-   **这一步实际上是在再看老师讲解的对不对，对了才保留。

**7. 生成候选演示: 记录解题过程 (记录下有效的解题步骤生成候选演示)**

-   **如果生成被认为是成功的（通过了 `metric` 阈值或默认情况下），** 则使用 `trace`（教师步骤的记录）来创建演示。
-   **创建 `dspy.Example` 对象:** `trace` 中的每个步骤都被转换为一个 `dspy.Example` 对象。此对象封装了该特定步骤中使用的预测器、它的输入和输出。
    -   `predictor`:  记录了解题步骤中使用的模块。
    -   `inputs`: 记录了输入给这个模块的数据。
    -   `outputs`: 记录了这个模块输出的结果。
-   **“然后把这些卡片按照步骤的顺序装订起来，形成一个完整的解题过程 (demonstration)”。**  这些 `dspy.Example` 对象按照推理的顺序组合起来，就形成了一个完整的演示。
-   **“我们把这些卡片称为“候选演示”。”**  之所以称为“候选”，是因为这些演示还需要经过筛选（步骤 9）才能最终被添加到 `demos` 中。
-   **`augmented` 标志设置为 `True`，以指示这是一个自举示例，表示这是通过自举的方式得到的, 而不是一开始就有的。重**

**9. 筛选候选演示: 挑选精华**

- **如果你的讲解过程 (trace) 中, 同一种方法 (predictor) 你记录了好几种不同的解题步骤 (demonstration),** 你就需要从中挑选出你认为最好的一种。
    - **例如：** 你在讲解“鸡兔同笼”问题时，对于“假设法”这个步骤，你可能尝试了“假设全是鸡”、“假设全是兔”等不同的假设方法, 并把这些尝试都记录下来了. 这时候，你就需要从中选择一个你认为最容易理解、最有效的解题步骤。
    - **怎么选呢？**  `BootstrapFewShot` 使用了一种简单的方法：根据这些解题步骤的内容计算出一个哈希值，然后根据这个哈希值创建一个随机数生成器。这样可以保证每次运行程序时, 对于相同的解题步骤, 都会选择相同的一个. 然后，以 50% 的概率选择除了最后一个步骤之外的其他步骤，以 50% 的概率选择最后一个步骤。这种方法既保证了一定的稳定性, 又引入了一些随机性, 有助于提高演示的多样性.

**10.  训练学生 (在 `_train` 中)**

*   自举过程完成后，使用以下组合训练学生模型：
    *   **自举演示:** 教师模型生成的演示。
    *   **标注演示（如果有）:** 来自 `trainset` 的具有正确答案的示例。
*   学生模型中的预测器将使用这些演示进行更新。在进行上下文学习时，演示会被放入提示中。

**11. 迭代改进:  不断优化**

- 当你再次讲解同一道题目, 或者类似的题目时，你会参考更新后的解题技巧和例题 (`demos`)。由于 `demos` 中包含了之前几轮迭代中生成的演示，因此你的讲解可能会受到这些演示的影响，从而产生不同的讲解思路和步骤，或者说产生不同的预测结果和不同的推理轨迹。这样，通过不断地迭代, 你讲解的质量就会越来越高, 学生也能学到更多、更好的解题方法, 从而提高解题能力。

**总结:**

自举迭代生成演示的流程就像是一位老师通过不断地尝试讲解、评估学生的解答、记录解题步骤、总结解题技巧，最终帮助学生掌握解题方法的过程。

-   **无标签数据利用:** 老师利用大量的习题 (无标签数据) 进行讲解，并通过记录解题步骤来生成演示。
-   **迭代改进:** 通过多轮讲解 (迭代)，老师不断优化自己的讲解方法 (预测)，并更新解题技巧 (demos)，从而提高学生的学习效果。
-   **关键:**  这个过程的关键在于，老师能够根据学生的解答情况 (评估预测结果) 来判断自己的讲解是否有效，并记录下有效的解题步骤 (生成候选演示)，最终总结出精华的解题技巧 (更新演示)。


In [47]:
from dspy.teleprompt import BootstrapFewShot
# optimizer.compile() 训练 你的模型 (修改提示)。
# Evaluate 测试 你的模型 (评估性能)。


# 使用 BootstrapFewShot 优化器
optimizer = BootstrapFewShot(metric=gsm8k_metric, max_bootstrapped_demos=4, max_labeled_demos=16, max_rounds=5, teacher_settings=dict(lm=turbo))
print("Bootstrap 优化器设置:", optimizer)

##optimizer.compile(): 负责使用训练集 (trainset) 和可选的验证集 (valset) 优化 你的 Module (例如 CoT 模块)。它会根据你选择的 Teleprompter (例如 BootstrapFewShot) 的策略来调整 Module 的内部参数，例如 Predictor 中的 demos。
optimized_cot = optimizer.compile(cot.deepcopy(), trainset=trainset[:50])

# 使用优化后的模型进行评估
# Evaluate: 负责使用一个独立的测试集 (如devset=devset[:20]) 评估 优化后的 Module 的性能。它会使用你指定的 metric (如 gsm8k_metric) 来计算模型在测试集上的表现。
evaluate = Evaluate(devset=devset[:20], metric=gsm8k_metric, num_threads=NUM_THREADS, display_progress=True, display_table=20,provide_traceback=True)
print("优化后评估结果:")
evaluate(optimized_cot)



Bootstrap 优化器设置: <dspy.teleprompt.bootstrap.BootstrapFewShot object at 0x00000236FBFCB170>


 34%|███▍      | 17/50 [00:01<00:03, 10.75it/s]


Bootstrapped 4 full traces after 17 examples for up to 5 rounds, amounting to 71 attempts.
优化后评估结果:
Average Metric: 8.00 / 20 (40.0%): 100%|██████████| 20/20 [00:00<00:00, 1471.48it/s]

2024/12/20 19:44:44 INFO dspy.evaluate.evaluate: Average Metric: 8 / 20 (40.0%)


,question,gold_reasoning,example_answer,reasoning,pred_answer,gsm8k_metric
0,"20 birds migrate on a seasonal basis from one lake to another, sea...",The birds' flying distance between Lake Jim through lake Disney to...,2200,To find the combined distance all of the birds have traveled in th...,4400,
1,"Wendy went to the dentist for a cleaning, two fillings, and a toot...",Wendy’s dentist bill was 5 * $120 = $<<5*120=600>>600. She got two...,290,Let's denote the cost of a filling as x. The total bill is five ti...,300,
2,Karen is packing her backpack for a long-distance hike. She packs ...,First find the weight of food Karen eats every hour: 2 pounds * 1/...,34,"To find out how much Karen is carrying after six hours, we need to...",33.33,
3,Rita is reading a five-chapter book with 95 pages. Each chapter ha...,Let P be the number of pages in the first chapter. The second chap...,13,"To find out how many pages the first chapter has, we need to work ...",13,✔️ [True]
4,Benjamin collects 6 dozen eggs a day. Carla collects 3 times the n...,Carla:3(6)=18 dozen eggs Trisha: 6-4=<<6-4=2>>2 dozen eggs Total: ...,26,Not supplied for this particular example.,18,
5,Roy spends 2 hours on sports activities in school every day. He go...,He goes to.school 5 days a week so if he misses 2 days within the ...,6,To find out how many hours Roy spent on sports in school that week...,7,
6,Cameron is printing her thesis in the school library and has 400 A...,"If she had 400 papers, the total number of documents she separated...",240,"To find out how many papers are invalid, we need to calculate 40% ...",240,✔️ [True]
7,Burt spent $2.00 on a packet of basil seeds and $8.00 on potting s...,He spent $2.00 on seeds and $8.00 on soil for a total of 2+8 = $<<...,90,"To find out Burt's net profit, we need to calculate his total reve...",90,✔️ [True]
8,Martha's cat catches 3 rats and 7 birds. Cara's cat catches 3 less...,First find the total number of animals Martha's cat catches: 3 rat...,47,Martha's cat catches 3 rats and 7 birds. Five times as many animal...,20,
9,Trey is raising money for a new bike that costs $112. He plans to ...,He needs to sell 112 bracelets because 112 divided by 1 equals <<1...,8,To find out how many bracelets Trey needs to sell per day on avera...,56,


40.0

In [14]:
# 查看优化前的模型中的示例（Demos）
for predictor in cot.predictors():
    print(f"Predictor: {predictor}")
    print("Demos:")
    for demo in predictor.demos:
        print(f"{demo}")
    print("\n")

Predictor: Predict(StringSignature(question -> reasoning, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))
Demos:




In [15]:
# 查看优化后的模型中的示例（Demos）
for predictor in optimized_cot.predictors():
    print(f"Predictor: {predictor}")
    print("Demos:")
    for demo in predictor.demos:
        print(f"{demo}")
    print("\n")


Predictor: Predict(StringSignature(question -> reasoning, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))
Demos:
Example({'augmented': True, 'question': 'A third of the contestants at a singing competition are female, and the rest are male. If there are 18 contestants in total, how many of them are male?', 'reasoning': 'Not supplied for this particular example.', 'answer': '12'}) (input_keys=None)
Example({'augmented': True, 'question': 'Megan pays $16 for a shirt that costs $22 bef

In [48]:
dspy.inspect_history(n=1)





[2024-12-20T19:44:44.882812]

System message:

Your input fields are:
1. `question` (str)

Your output fields are:
1. `reasoning` (str)
2. `answer` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `answer`.


User message:

This is an example of the task, though some input or output fields are not supplied.

[[ ## question ## ]]
Audrey is 7 years older than Heracles. In 3 years, Audrey will be twice as old as Heracles is now. How old is Heracles now?

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## answer ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Assistant message:

[[ ## reasoning ## ]]
Not supplied for this particul

-  (Bootstrapped 4 full traces after 17 examples for up to 5 rounds, amounting to 71 attempts)
先复习一下自举生成演示 (bootstrap demonstrations)的概念: 
    - 指的是利用教师模型对示例进行预测，并将预测过程中的步骤（predictor, inputs, outputs【包含 thought 和 answer 字段】）记录下来，作为新的演示添加到 demos 中。
- **`Bootstrapped 4 full traces`:**  表示成功生成了 4 个完整的自举演示（或者说，4 个完整的推理轨迹 问题-推理过程-答案）。
- **`after 17 examples`:**  表示在遍历了 17 个训练示例之后，成功生成了 4 个完整的自举演示。
- **`for up to 5 rounds`:**  表示每个示例最多尝试生成自举演示的次数是 5 次（即 `max_rounds=5`）。如果某一轮成功生成了满足条件的自举演示（通过了 metric 的评估），那么针对该示例的迭代就会提前结束。如果 5 轮结束后都没有生成有效的自举演示，那么就放弃该示例，继续处理下一个示例。
- **`amounting to 71 attempts`:**  表示总共尝试了 71 次自举操作（包括成功的和失败的）。

**这个例子说明了：**

- 并非所有的尝试都能成功生成自举演示。在 71 次尝试中，只有 4 次成功了。
- 通过多轮迭代和评估指标的筛选，可以不断改进演示的质量。虽然只有 4 个自举演示被生成, 但这 4 个演示是经过多轮迭代和评估指标筛选出来的, 因为选用的老师模型就是他自己，它们代表了当前模型能力下, 比较高质量的推理过程. 随着迭代次数的增加形成了正向循环, demos 质量提高 --> 教师模型能力提高 --> 生成更高质量的演示 --> demos 质量进一步提高。`demos` 中的演示质量会不断提高, 从而生成更高质量的演示.

怎么优化呢？

调整 `BootstrapFewShot` 的参数是一个迭代的过程，你需要根据自举成功率、评估结果、计算资源和训练时间等因素进行权衡。建议从一个合理的参数组合开始，然后根据实验结果逐步调整，找到最佳的参数配置。没有一成不变的最佳参数，只有最适合你的任务和数据的参数。

**核心参数及其调整策略:**

1. **`metric`**:

    *   **含义**: 用于评估模型预测结果的函数。通常不需要调整，但要确保 `gsm8k_metric` 函数的逻辑正确且符合你的评估需求。
    *   **示例**: 保持不变 (除非你需要修改评估逻辑)。

    ```python
    optimizer = BootstrapFewShot(metric=gsm8k_metric, ...) 
    # 保持不变
    ```

2. **`metric_threshold`**:

    *   **含义**:  当 `metric` 返回数值时，用于判断是否接受自举生成的示例的阈值。只有当 `metric` 的返回值大于或等于该阈值时，自举示例才会被接受。
    *   **调整策略**:
        *   **降低阈值**: 如果自举成功率较低 (例如，`Bootstrapped 4 full traces after 17 examples` 这种情况)，可以尝试降低阈值，放宽接受自举示例的标准。这可能会增加自举示例的数量，但也可能降低示例的质量。
        *   **提高阈值**: 如果你希望获得更高质量的自举示例，可以尝试提高阈值。这可能会减少自举示例的数量，但可以提高示例的质量。
        *   **注意**:  如果你的 `metric` 返回的是布尔值 (True/False)，则不需要设置 `metric_threshold`。(本次因为返回的是布尔值就不用设置)
    *   **示例**:
        *   假设你其他项目里 `metric` 返回的是预测答案与真实答案之间的相似度分数 (0-1 之间)，并且你当前的 `metric_threshold` 为 0.8。
        *   **降低阈值**:

            ```python
            optimizer = BootstrapFewShot(metric=gsm8k_metric,
             metric_threshold=0.6, ...)  # 降低阈值到 0.6
            ```

        *   **提高阈值**:

            ```python
            optimizer = BootstrapFewShot(metric=gsm8k_metric, 
            metric_threshold=0.9, ...)  # 提高阈值到 0.9
            ```

3. **`max_bootstrapped_demos`**:

    *   **含义**: 生成的自举示例的最大数量。
    *   **调整策略**:
        *   **增加数量**: 如果自举成功率较低，或者你希望使用更多的自举示例，可以尝试增加 `max_bootstrapped_demos`。
        *   **减少数量**: 如果你担心生成的自举示例质量不高，或者希望减少训练时间，可以尝试减少 `max_bootstrapped_demos`。
    *   **示例**:
        *   **增加数量**:

            ```python
            optimizer = BootstrapFewShot(metric=gsm8k_metric, 
            max_bootstrapped_demos=8, ...)  # 增加到 8 个
            ```

        *   **减少数量**:

            ```python
            optimizer = BootstrapFewShot(metric=gsm8k_metric, 
            max_bootstrapped_demos=2, ...)  # 减少到 2 个
            ```

4. **`max_labeled_demos`**:

    *   **含义**:  标注数据的最大数量。
    *   **调整策略**:
        *   **增加数量**: 如果你的标注数据比较多，并且质量较高，可以尝试增加 `max_labeled_demos`。
        *   **减少数量**: 如果你的标注数据比较少，或者质量较低，可以尝试减少 `max_labeled_demos`。
    *   **示例**:
        *   **增加数量**:

            ```python
            optimizer = BootstrapFewShot(metric=gsm8k_metric, 
            max_labeled_demos=32, ...)  # 增加到 32 个
            ```

        *   **减少数量**:

            ```python
            optimizer = BootstrapFewShot(metric=gsm8k_metric, 
            max_labeled_demos=8, ...)  # 减少到 8 个
            ```

5. **`max_rounds`**:

    *   **含义**: 自举迭代的最大轮次数。
    *   **调整策略**:
        *   **增加轮次**: 如果自举成功率较低，可以尝试增加 `max_rounds`，进行更多轮次的自举迭代。
        *   **减少轮次**: 如果你希望减少训练时间，可以尝试减少 `max_rounds`。
    *   **示例**:
        *   **增加轮次**:

            ```python
            optimizer = BootstrapFewShot(metric=gsm8k_metric, 
            max_rounds=10, ...)  # 增加到 10 轮
            ```

        *   **减少轮次**:

            ```python
            optimizer = BootstrapFewShot(metric=gsm8k_metric, 
            max_rounds=3, ...)  # 减少到 3 轮
            ```

6. **`teacher_settings`**:

    *   **含义**: 老师模型的配置参数。
    *   **调整策略**:
        *   **使用更强大的模型**: 如果你的计算资源允许，可以尝试使用更强大的模型作为老师模型。
        *   **调整 temperature**: 你可以调整老师模型的 `temperature` 参数，以控制生成的多样性。
    *   **示例**:
        *   **使用更强大的模型**:

            ```python
            turbo_teacher = dspy.LM('ollama_chat/llama3:7b', 
            api_base='http://localhost:11434', api_key='')
            optimizer = BootstrapFewShot(metric=gsm8k_metric, 
            teacher_settings=dict(lm=turbo_teacher), ...)
            ```

        *   **调整 temperature**:

            ```python
            optimizer = BootstrapFewShot(metric=gsm8k_metric, 
            teacher_settings=dict(lm=turbo, temperature=0.8), ...) 
             # 增加 temperature 到 0.8
            ```

7. **`max_errors`**:
    *   **含义**: 允许的最大错误次数。如果在自举过程中发生超过 `max_errors` 次错误，程序将提前终止。
    *   **调整策略**:
        *   **增加错误次数**: 如果你不希望程序因为一些错误而过早终止，可以尝试增加 `max_errors`。
        *   **减少错误次数**: 如果你希望程序在遇到错误时更早地停止，可以尝试减少 `max_errors`。
    *   **示例**:
        *   **增加错误次数**:

            ```python
            optimizer = BootstrapFewShot(metric=gsm8k_metric, max_errors=10, ...)
            ```
        *   **减少错误次数**:

            ```python
            optimizer = BootstrapFewShot(metric=gsm8k_metric, max_errors=3, ...)
            ```


In [17]:
# 之前的 BootstrapFewShot 优化器
# optimizer = BootstrapFewShot(metric=gsm8k_metric, max_bootstrapped_demos=4, max_labeled_demos=16, max_rounds=5, teacher_settings=dict(lm=turbo))
teacher = dspy.LM('ollama_chat/llama3.2:3b', api_base='http://localhost:11434', api_key='')

In [18]:

# 使用 BootstrapFewShot 优化器
optimizer = BootstrapFewShot(
    metric=gsm8k_metric,
    max_bootstrapped_demos=5,  # 增加自举示例数量
    max_labeled_demos=6,# 减少数据集示例
    max_rounds=6,  # 增加迭代轮次
    teacher_settings=dict(lm=teacher),
    max_errors=5
)

# 执行优化
optimized_new_cot = optimizer.compile(cot.deepcopy(), trainset=trainset[:50])
evaluate = Evaluate(devset=devset[:30], metric=gsm8k_metric, num_threads=NUM_THREADS, display_progress=True, display_table=20,provide_traceback=True)
# 使用优化后的模型进行评估
print("优化后评估结果:")
evaluate(optimized_new_cot)

 10%|█         | 5/50 [00:00<00:00, 169.13it/s]


Bootstrapped 5 full traces after 5 examples for up to 6 rounds, amounting to 5 attempts.
优化后评估结果:
Average Metric: 13.00 / 30 (43.3%): 100%|██████████| 30/30 [00:00<00:00, 163.75it/s]

2024/12/20 16:09:34 INFO dspy.evaluate.evaluate: Average Metric: 13 / 30 (43.3%)


,question,gold_reasoning,example_answer,reasoning,pred_answer,gsm8k_metric
0,"20 birds migrate on a seasonal basis from one lake to another, sea...",The birds' flying distance between Lake Jim through lake Disney to...,2200,"The first season, 20 birds fly from lake Jim to lake Disney. The d...",2200,✔️ [True]
1,"Wendy went to the dentist for a cleaning, two fillings, and a toot...",Wendy’s dentist bill was 5 * $120 = $<<5*120=600>>600. She got two...,290,Let's denote the cost of a filling as x. Then the total bill is 5x...,$140 + 4x,
2,Karen is packing her backpack for a long-distance hike. She packs ...,First find the weight of food Karen eats every hour: 2 pounds * 1/...,34,"First, calculate how much water Karen drinks per hour: 2 pounds/ho...",-6.08,
3,Rita is reading a five-chapter book with 95 pages. Each chapter ha...,Let P be the number of pages in the first chapter. The second chap...,13,Let's denote the number of pages in the first chapter as x. Since ...,19,
4,Benjamin collects 6 dozen eggs a day. Carla collects 3 times the n...,Carla:3(6)=18 dozen eggs Trisha: 6-4=<<6-4=2>>2 dozen eggs Total: ...,26,Benjamin collects 6 dozen eggs a day. To find out how many dozens ...,24,
5,Roy spends 2 hours on sports activities in school every day. He go...,He goes to.school 5 days a week so if he misses 2 days within the ...,6,To find out how many hours Roy spent on sports in school that week...,6,✔️ [True]
6,Cameron is printing her thesis in the school library and has 400 A...,"If she had 400 papers, the total number of documents she separated...",240,"First, find 40% of 400: 0.4 x 400 = 160 This means that 160 papers...",160,
7,Burt spent $2.00 on a packet of basil seeds and $8.00 on potting s...,He spent $2.00 on seeds and $8.00 on soil for a total of 2+8 = $<<...,90,"To find Burt's total revenue, we need to multiply the number of ba...",$92,
8,Martha's cat catches 3 rats and 7 birds. Cara's cat catches 3 less...,First find the total number of animals Martha's cat catches: 3 rat...,47,"Martha's cat catches 3 rats and 7 birds. First, find five times as...",47,✔️ [True]
9,Trey is raising money for a new bike that costs $112. He plans to ...,He needs to sell 112 bracelets because 112 divided by 1 equals <<1...,8,To find out how many bracelets Trey needs to sell per day on avera...,$8,✔️ [True]


43.33

为什么自举比few shot效果更好？加大了自举示例占总例子比例后正确率提升，因为是让大模型自己想问题，自己思考，可能比我们的额数据集更符合大模型解答的规律。

### 优化器：BootstrapFewShotWithRandomSearch



**`BootstrapFewShot` (单个老师，固定教学方法)**

1. **一个老师：**  只有一个教师模型。
2. **一套固定的教学方法：** 使用固定的参数（例如 `max_bootstrapped_demos`）进行自举。
3. **一套演示集合：**  对每个训练示例，教师模型尝试 “讲解” 并生成演示，最多尝试 `max_rounds` 次，最后生成一套演示集合
4. **生成演示：**  将“讲解”过程中满足条件的步骤记录下来，作为演示。
5. **更新演示：**  将生成的演示添加到教师模型的 `demos` 属性中。
6. **学生学习：** 学生模型（可以与教师模型相同，也可以不同）在训练阶段使用这些演示进行学习。

**`BootstrapFewShotWithRandomSearch` (单个老师，多种教学方法)**

1. **一个老师：**  同样只有一个教师模型。但是, 这个老师会尝试不同的教学方法.
2. **多套教学方法 (通过随机搜索实现)：**
    -   通过**不同的随机种子 (seed)** 来控制不同的教学方法。例如，每个种子可以对应不同的：
        -   训练集打乱的顺序。
        -   每个预测器使用的自举演示的数量 (在 `min_num_samples` 和 `max_num_samples` 之间随机选择)。
    -   **`if seed == -3: ... elif seed == -2: ... elif seed == -1: ... else: ...`:**  根据不同的种子值，选择不同的优化策略：
        -   **`seed == -3`:**  零样本 (zero-shot) 学习，直接使用学生模型，不进行任何提示优化。
        -   **`seed == -2`:**  仅使用已标注的示例，使用 `LabeledFewShot` 优化器。
        -   **`seed == -1`:**  使用 `BootstrapFewShot` 优化器，但不打乱训练数据集。
        -   **`seed >= 0`:**  使用 `BootstrapFewShot` 优化器，并打乱训练数据集。根据种子值随机选择要生成的示例数量。
               
3. **多次尝试 (多个候选程序)：**
    -   对于**每一种教学方法**，都会创建一个**候选程序**。你可以把每个候选程序理解为使用一种特定教学方法进行教学后的学生.
    -   每个候选程序都会独立地经历 `BootstrapFewShot` 的自举过程，生成自己的演示集合。
4. **生成演示：**  每个候选程序都会生成自己的演示集合（与 `BootstrapFewShot` 相同）。
5. **更新演示：** 每个程序都会有自己的 `demos` 属性, 用于保存该程序生成的演示.
6. **评估和选择：**
    -   使用 `Evaluate` 类评估每个候选程序（即使用不同教学方法产生的学生）在验证集上的性能。
    -   **怎么找出最优的** 比较所有候选程序的性能，选择性能最好的那个作为最终的程序（即选择“教得最好”的那种教学方法产生的学生）。
    -   会将所有候选程序的信息, 包括评估分数, 演示集合等, 都记录到最佳程序的 `candidate_programs` 属性中.

**可以这样理解:**

-   `BootstrapFewShot` 就像一个老师用一种固定的方法教学生。
-   `BootstrapFewShotWithRandomSearch` 就像一个老师尝试了多种不同的教学方法，然后让这些学生参加考试 (评估)，最后选择考试成绩最好的那个学生所用的那套方法, 作为最终的教学方法。


**与 `BootstrapFewShot` 相比，`BootstrapFewShotWithRandomSearch` 具有以下优势：**

-   **更好的性能:** 通过探索多个候选程序，更有可能找到一个性能更好的程序。
-   **更鲁棒:**  不太依赖于特定的超参数设置, 因为它会尝试多种不同的设置.
-   **可解释性:**  通过 `candidate_programs` 属性可以了解不同的自举策略对性能的影响.

**当然，`BootstrapFewShotWithRandomSearch` 也有一些缺点：**

-   **计算成本更高:** 需要生成和评估多个候选程序，计算成本更高。
-   **需要更多的超参数调整:**  需要调整更多的超参数, 例如 `num_candidate_programs`、`min_num_samples`、`max_num_samples` 等.

`BootstrapFewShotWithRandomSearch` 是一种更强大但也更复杂的自举方法, 它通过引入随机搜索来提高模型的性能.

**超参数变化:**  
  
`BootstrapFewShotWithRandomSearch` 引入了一些新的超参数来控制搜索过程, 例如:  
  
-   `num_candidate_programs`: 要生成的候选程序数量（教学方法）.  
-   `stop_at_score`: 搜索过程中提前停止搜索的分数阈值.如果设置得太低，可能会错过更好的候选程序；如果设置得太高，则可能永远无法达到阈值，导致搜索所有候选程序。  
-   `min_num_samples` 和 `max_num_samples`: 控制每个 predictor 使用的自举演示数量的范围.

-   注意**`num_candidate_programs` 控制的是 `seed` 值大于等于 0 的情况的数量**。如配置 `num_candidate_programs=2`，这意味着 `seed` 将取值 0 和 1，对应于两种不同的随机打乱和示例数量的 `BootstrapFewShot` 配置。

-   **除了 `seed` 大于等于 0 的情况，代码还固定地考虑了另外三种情况：**
    -   **`seed = -3` (零样本):**  不使用任何提示，直接评估原始模型。
    -   **`seed = -2` (仅使用已标注示例):** 使用 `LabeledFewShot` 优化器。
    -   **`seed = -1` (未打乱的 `BootstrapFewShot`):** 使用 `BootstrapFewShot` 优化器，但不打乱训练集。

-   因此，**总的候选程序数量 = `num_candidate_programs` (对应于 `seed` >= 0 的情况) + 3 (对应于 `seed` 为 -3, -2, -1 的固定情况)**。

所以如果`num_candidate_programs` 为 2，所以总的候选程序数量为 2 + 3 = 5。会循环尝试 `seed` 值为 -3, -2, -1, 0, 1，并在最后打印 "5 candidate programs found."有5个候选程序。

In [20]:
config = dict(max_bootstrapped_demos=3, max_labeled_demos=2, num_candidate_programs=2, num_threads=NUM_THREADS,max_errors=7,max_rounds=5,teacher_settings=dict(lm=turbo))
optimizer_BootstrapFewShotWithRandomSearch = BootstrapFewShotWithRandomSearch(metric=gsm8k_metric, **config)
cot_bs = optimizer_BootstrapFewShotWithRandomSearch.compile(cot.deepcopy(), trainset=trainset[:10], valset=devset[20:25])

evaluate = Evaluate(devset=devset[60:70], metric=gsm8k_metric, num_threads=NUM_THREADS, display_progress=True, display_table=20,provide_traceback=True)

evaluate(optimized_new_cot)

Going to sample between 1 and 3 traces per predictor.
Will attempt to bootstrap 2 candidate sets.
Average Metric: 0.00 / 5 (0.0%): 100%|██████████| 5/5 [00:00<00:00, 363.98it/s]

2024/12/20 16:09:37 INFO dspy.evaluate.evaluate: Average Metric: 0 / 5 (0.0%)



New best score: 0.0 for seed -3
Scores so far: [0.0]
Best score so far: 0.0
Average Metric: 1.00 / 5 (20.0%): 100%|██████████| 5/5 [00:38<00:00,  7.70s/it]

2024/12/20 16:10:16 INFO dspy.evaluate.evaluate: Average Metric: 1 / 5 (20.0%)



New best score: 20.0 for seed -2
Scores so far: [0.0, 20.0]
Best score so far: 20.0


...

2024/12/20 16:15:07 INFO dspy.evaluate.evaluate: Average Metric: 1 / 5 (20.0%)



Scores so far: [0.0, 20.0, 40.0, 20.0, 20.0]
Best score so far: 40.0
5 candidate programs found.
Average Metric: 5.00 / 10 (50.0%): 100%|██████████| 10/10 [01:56<00:00, 11.65s/it]

2024/12/20 16:17:04 INFO dspy.evaluate.evaluate: Average Metric: 5 / 10 (50.0%)


,question,gold_reasoning,example_answer,reasoning,pred_answer,gsm8k_metric
0,Jose owns a swimming pool. He charges $3 for kids and twice that a...,Jose can earn $3 x 8 = $<<3*8=24>>24 per day for kids. He charges ...,588,Jose charges $3 for kids and twice that amount ($6) for adults. So...,$588,✔️ [True]
1,"To improve her health, Mary decides to drink 1.5 liters of water a...",Each liter of water holds 1000 mL. Mary's goal is to drink 1.5 lit...,6,To find out how many glasses of water Mary should drink per day to...,6,✔️ [True]
2,An airplane was flying from California to Virginia. The flight sta...,They started with 124 passengers. 58 got off and 24 got on so 124-...,67,"First, calculate the number of passengers who got off at Texas: 58...",97,
3,Frank and Bill have $42 and they bought 3 large pizzas with the mo...,Frank spent 11*3 = <<11*3=33>>33 dollars on pizza. Frank gives 42-...,39,Frank bought 3 pizzas at $11 each. So he spent a total of: 3 x $11...,$39,✔️ [True]
4,Cathy and Chris got summer jobs at the cake shop and were supposed...,There are 4 weeks in a month so Cathy and Chris were supposed to w...,180,"Cathy worked 20 hours per week for 4 weeks in a month. So, she wor...",80,
5,A store sold a certain brand of jeans for $40. They only have few ...,Two pairs of jeans cost $40 x 2 = $<<40*2=80>>80. A discount of $8...,112,"First, calculate the discount on two pairs of jeans: 10% of $80 = ...",$96,
6,"Every year in January, Natalia, the librarian, tidies up the books...",Let’s first calculate the total number of items: 145 + 271 + 419 +...,116,"To find out how many crates Natalia needs to put her books in, we ...",116,✔️ [True]
7,"Ellen is baking bread. It takes 3 hours to rise 1 ball of dough, a...",Ellen takes 4*3=<<4*3=12>>12 hours in total to rise all balls of d...,20,"To find the total time it takes to bake all 4 balls of dough, we n...",80,
8,Eddie can bake 3 pies a day. His sister can bake 6 pies while his ...,Eddie bakes 3 x 7 = <<3*7=21>>21 pies for 7 days. His sister bakes...,119,"To find out how many pies they will be able to bake in 7 days, we ...",119,✔️ [True]
9,Florida is starting a promotion where every car that arrives gets ...,The oranges would've cost $6 because 4 x 1.5 = <<4*1.5=6>>6 The pr...,40,The family had planned to spend $15 in total. Since they got 4 ora...,60%,


50.0

配置如下：

```python
config = dict(
    max_bootstrapped_demos=3,  # 每个 predictor 最多生成 3 个示例
    max_labeled_demos=2,  # 最多使用 2 个已标注示例
    num_candidate_programs=2,  # 生成 2+3 个候选程序
    num_threads=NUM_THREADS,  # 使用 NUM_THREADS 个线程进行评估
    max_errors=7,  # 评估时最多允许 7 个错误
    max_rounds=5,  # BootstrapFewShot 的最大迭代轮数为 5
    teacher_settings=dict(lm=turbo)  # teacher 模型的设置为 {lm: turbo}
)
```

`compile` 方法的调用如下：

```python
cot_bs = optimizer_BootstrapFewShotWithRandomSearch.compile(
    cot.deepcopy(),  # 待优化的学生模型（CoT 模型的深拷贝）
    trainset=trainset[:10],  # 训练集为 trainset 的前 10 个样本
    valset=devset[20:25]  # 验证集为 devset 的第 21 到 25 个样本（索引从 0 开始）
)
```



**1. 初始化 `BootstrapFewShotWithRandomSearch` 实例:**

```
将为每个预测器采样 1 到 3 个轨迹。
将尝试引导 2 个候选程序集。
```
  **`将为每个预测器采样 1 到 3 个轨迹。`**:  `min_num_samples` 为 1，`max_num_samples` (即 `max_bootstrapped_demos`) 为 3。对于 `seed` 大于等于 0 的情况，每个 `predictor` 将随机采样 1 到 3 个示例。
-   **`将尝试引导 2 个候选程序集。`**: `num_candidate_programs` 为 2，这意味着循环将尝试 `seed` 值为 -3, -2, -1, 0, 1 的情况，并在循环的最后打印"5 candidate programs found."。


**2. `compile` 方法开始执行:**

-   `trainset` 被设置为 `trainset[:10]`，即前 10 个训练样本。
-   `valset` 被设置为 `devset[20:25]`，即第 21 到 25 个验证样本。
-   `scores`, `all_subscores`, `score_data` 列表被初始化为空列表。

**3. 循环遍历种子 (seed):**

-   循环将从 `seed = -3` 开始，一直到 `seed = num_candidate_programs - 1 = 2 - 1 = 1`，即 `seed` 的取值为 -3, -2, -1, 0, 1。
-   `restrict` 参数未指定，因此不会跳过任何种子。

**4. 针对每个种子生成候选程序并评估:**

   **a) `seed = -3` (零样本):**

   -   `program` 设置为 `student.reset_copy()`，即 `cot` 模型的一个重置副本，不使用任何提示。
   -   使用 `Evaluate` 实例评估 `program` 在 `valset` 上的性能。
   -   评估结果（`score` 和 `subscores`）将被添加到 `all_subscores` 列表。
   -   进行断言感知优化（如果适用）。
   -   如果当前分数是第一个分数，或者大于之前的最高分数，则将 `program` 设置为 `best_program`。
   -   将当前分数添加到 `scores` 列表。
   -   将 `(score, subscores, seed, program)` 元组添加到 `score_data` 列表。
   -   打印相关分数信息：

       ```
       新的最佳分数：[某个具体分数]，种子为 -3
       目前的分数：[[某个具体分数]]
       目前的最高分数：[某个具体分数]
       ```
- INFO dspy.evaluate.evaluate: Average Metric: 0 / 5 (0.0%)
-   **`New best score: 0.0 for seed -3`**:  由于这是第一个候选程序，因此它的分数 0.0 被认为是新的最佳分数。
-   **`Scores so far: [0.0]`**: 目前为止的分数列表为 `[0.0]`。
-   **`Best score so far: 0.0`**: 目前的最佳分数为 0.0。


   **b) `seed = -2` (仅使用已标注示例):**

   -   创建一个 `LabeledFewShot` 优化器，`k=self.max_labeled_demos=2`。
   -   使用 `LabeledFewShot.compile` 方法，用 `trainset_copy`（即 `trainset[:10]` 的副本）中的最多 2 个已标注示例来编译 `student`（即 `cot` 的副本），生成 `program`。
   -   后续评估流程与 `seed = -3` 相同。假设 `seed=-2` 得到了比`-3`更高的分数，那么打印的信息会类似于（已翻译成中文）：
     
```

2024/12/20 16:10:16 INFO dspy.evaluate.evaluate: Average Metric: 1 / 5 (20.0%)

New best score: 20.0 for seed -2
Scores so far: [0.0, 20.0]
Best score so far: 20.0
```


   **c) `seed = -1` (未打乱的 `BootstrapFewShot`):**

   -   创建一个 `BootstrapFewShot` 优化器，其参数如下：
       -   `max_bootstrapped_demos=self.max_num_samples=3` (因为在`__init__`中，`max_num_samples`被初始化成`max_bootstrapped_demos`)
       -   `max_labeled_demos=2`
       -   `teacher_settings=dict(lm=turbo)`
       -   `max_rounds=5`
       -   `max_errors=7`
   -   使用 `BootstrapFewShot.compile` 方法，用 `trainset_copy` 和 `teacher`（这里 `teacher` 为 `None`）编译 `student`，生成 `program`。
   -   `BootstrapFewShot` 会尝试使用最多 3 个示例进行引导，最多迭代 5 轮，直到错误数达到 7 或达到其他停止条件。
   -   后续评估流程与之前相同。
   -   **`2024/12/20 16:12:37 INFO dspy.evaluate.evaluate: Average Metric: 2 / 5 (40.0%)`**: 日志信息确认平均指标为 2。
-   **`New best score: 40.0 for seed -1`**: 由于 2.00 / 5 = 0.4 = 40.0% > 20.0，因此 40.0 被认为是新的最佳分数。
-   **`Scores so far: [0.0, 20.0, 40.0]`**: 目前的分数列表为 `[0.0, 20.0, 40.0]`。
-   **`Best score so far: 40.0`**: 目前的最佳分数为 40.0。

   **d) `seed = 0` (打乱的 `BootstrapFewShot`):**

   -   使用 `random.Random(0)` 对 `trainset_copy` 进行打乱。
   -   使用 `random.Random(0).randint(1, 3)` 生成一个随机数 `size`，表示要生成的示例数量（范围为 1 到 3）。
   -   创建一个 `BootstrapFewShot` 优化器，其 `max_bootstrapped_demos` 设置为 `size`，其他参数与 `seed = -1` 时相同。
   -   使用 `BootstrapFewShot.compile` 方法生成 `program`。
   -   后续评估流程与之前相同。
   - -   **`2024/12/20 16:13:42 INFO dspy.evaluate.evaluate: Average Metric: 1 / 5 (20.0%)`**: 日志信息确认平均指标为 1。
-   **`Scores so far: [0.0, 20.0, 40.0, 20.0]`**:  目前的分数列表为 `[0.0, 20.0, 40.0, 20.0]`。
-   **`Best score so far: 40.0`**:  最佳分数仍然是 40.0，因为当前分数 20.0 没有超过它。

   **e) `seed = 1` (打乱的 `BootstrapFewShot`):**

   -   使用 `random.Random(1)` 对 `trainset_copy` 进行打乱。
   -   使用 `random.Random(1).randint(1, 3)` 生成一个随机数 `size`。
   -   创建一个 `BootstrapFewShot` 优化器，其 `max_bootstrapped_demos` 设置为 `size`，其他参数与 `seed = -1` 时相同。
   -   使用 `BootstrapFewShot.compile` 方法生成 `program`。
   -   后续评估流程与之前相同。
   -  **`2024/12/20 16:15:07 INFO dspy.evaluate.evaluate: Average Metric: 1 / 5 (20.0%)`**: 日志信息确认平均指标为 1。
-   **`Scores so far: [0.0, 20.0, 40.0, 20.0, 20.0]`**: 目前的分数列表为 `[0.0, 20.0, 40.0, 20.0, 20.0]`。
-   **`Best score so far: 40.0`**: 最佳分数仍然是 40.0。

**5. 循环结束:**

-   所有种子遍历完毕后，循环结束。
-   `best_program` 将被设置为所有候选程序中在 `valset` 上得分最高的程序。

**6. 附加候选程序信息:**

-   `score_data`（包含所有候选程序的分数、子分数、种子和程序）将被附加到 `best_program.candidate_programs` 属性中。
-   `best_program.candidate_programs` 将根据分数进行降序排序。

**7. 返回最佳程序:**

-   打印找到的候选程序数量（已翻译成中文）：

```
找到 5 个候选程序。
```

-   返回 `best_program`。

**总结:**

通过结合输出信息和代码流程分析，我们可以清晰地看到 `BootstrapFewShotWithRandomSearch` 在不同种子下的运行情况，以及每个候选程序的评估结果。最终，`seed = -1` 的未打乱的 `BootstrapFewShot` 生成的程序表现最好，其在验证集上的指标值为 40.0%。最终返回的 `best_program` 是这 5 个程序中在 `valset` 上得分最高的程序。
在实际评估中10道题对了5个
